In [2]:
# !conda install -c conda-forge folium=0.5.0 --yes
# !conda install -c conda-forge BeautifulSoup4 --yes
# !conda install -c conda-forge lxml --yes
# !conda install -c conda-forge geocoder --yes
# !conda install -c conda-forge geopy --yes

import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-2.8           |   py36h45558ae_2         628 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    pandas-1.0.3               |   py36h830a2c2_0        11.1 MB  conda-forge
    pysocks-1.7.1              |   py36h9f0ad1d_1          27 KB  conda-forge
    toolz-0.10.0               |             py_0          46 KB  conda-

In [3]:
df=pd.read_csv('zip_code_database.csv')
column_names=['zip','primary_city','acceptable_cities','county','latitude','longitude','irs_estimated_population_2015']
df = df[df['state']=='FL']
df = df[(df['county'] == 'Miami-Dade County') | (df['county'] == 'Broward County') | (df['county'] == 'Palm Beach County')]
df = df[df['type']=='STANDARD']
df = df[column_names]
df.reset_index(drop=True, inplace=True)
df = df.rename(columns={'irs_estimated_population_2015': 'population'})
print(df.head())

     zip primary_city              acceptable_cities             county  \
0  33004        Dania                    Dania Beach     Broward County   
1  33009   Hallandale  Hallandale Beach, Halndle Bch     Broward County   
2  33010      Hialeah                            NaN  Miami-Dade County   
3  33012      Hialeah                            NaN  Miami-Dade County   
4  33013      Hialeah                            NaN  Miami-Dade County   

   latitude  longitude  population  
0     26.05     -80.14       12800  
1     25.98     -80.14       31890  
2     25.83     -80.28       37940  
3     25.86     -80.29       62200  
4     25.86     -80.27       29190  


In [7]:
CLIENT_ID = 'LUZDEBPP24RGLIOMGTZ5RDHNWNMM0E5UFEQJEAPYB0NGAYXW'
CLIENT_SECRET = 'XTWLHTW4WJRGIHYTHL4MTUCHFMJFG5MZPACM5A2CNHBI1EUE'
VERSION = '20180604'

def getNearbyVenues(names, latitudes, longitudes):
    radius = 500
    LIMIT = 100
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']

    return (nearby_venues)

sfl_venues = getNearbyVenues(names=df['primary_city'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )
print(sfl_venues.head())

  Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0        Dania                  26.05                  -80.14   
1        Dania                  26.05                  -80.14   
2        Dania                  26.05                  -80.14   
3        Dania                  26.05                  -80.14   
4        Dania                  26.05                  -80.14   

                                               Venue  Venue Latitude  \
0                                        Conca D'oro       26.051541   
1  Jaxson's Ice Cream Parlour, Restaurant & Count...       26.049361   
2                                            Mundi's       26.051600   
3                                Paolo's 3's Company       26.052226   
4                                         Bake Shack       26.047426   

   Venue Longitude      Venue Category  
0       -80.137166  Italian Restaurant  
1       -80.143847      Ice Cream Shop  
2       -80.138026  Italian Restaurant  
3       -80.

In [2]:
# !conda install -c conda-forge dill --yes
import dill
# dill.dump_session('notebook_env.db')
dill.load_session('notebook_env.db')

In [3]:
print(sfl_venues.groupby('Neighborhood').count())

                    Neighborhood Latitude  Neighborhood Longitude  Venue  \
Neighborhood                                                               
Belle Glade                             4                       4      4   
Boca Raton                             76                      76     76   
Boynton Beach                          42                      42     42   
Dania                                  24                      24     24   
Deerfield Beach                         9                       9      9   
Delray Beach                          119                     119    119   
Fort Lauderdale                       424                     424    424   
Hallandale                             28                      28     28   
Hialeah                                36                      36     36   
Hollywood                              33                      33     33   
Homestead                              10                      10     10   
Jupiter     

In [4]:
sfl_onehot = pd.get_dummies(sfl_venues[['Venue Category']], prefix="", prefix_sep="")
sfl_onehot.drop(['Neighborhood'],axis=1,inplace=True)
sfl_onehot.insert(loc=0, column='Neighborhood', value=sfl_venues['Neighborhood'] )
print(sfl_onehot.shape)

(1579, 264)


In [5]:
sfl_grouped = sfl_onehot.groupby('Neighborhood').mean().reset_index()
print(sfl_grouped.head())

      Neighborhood  ATM  Accessories Store  Advertising Agency  Airport  \
0      Belle Glade  0.0                0.0                 0.0      0.0   
1       Boca Raton  0.0                0.0                 0.0      0.0   
2    Boynton Beach  0.0                0.0                 0.0      0.0   
3            Dania  0.0                0.0                 0.0      0.0   
4  Deerfield Beach  0.0                0.0                 0.0      0.0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0             0.0              0.0               0.0             0.000000   
1             0.0              0.0               0.0             0.039474   
2             0.0              0.0               0.0             0.071429   
3             0.0              0.0               0.0             0.000000   
4             0.0              0.0               0.0             0.000000   

   Arcade  ...  Video Game Store  Video Store  Vietnamese Restaurant  \
0     0.0  ...

In [8]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sfl_grouped['Neighborhood']

for ind in np.arange(sfl_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sfl_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.head())

      Neighborhood 1st Most Common Venue            2nd Most Common Venue  \
0      Belle Glade  Fast Food Restaurant  Southern / Soul Food Restaurant   
1       Boca Raton  Fast Food Restaurant                       Donut Shop   
2    Boynton Beach              Pharmacy              American Restaurant   
3            Dania    Italian Restaurant                   Breakfast Spot   
4  Deerfield Beach                  Pool                            Hotel   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0           Pizza Place     Fondue Restaurant     Fish & Chips Shop   
1        Sandwich Place                   Gym    Italian Restaurant   
2        Sandwich Place                 Diner        Cosmetics Shop   
3        Discount Store   Fried Chicken Joint      Asian Restaurant   
4  Gym / Fitness Center        Sandwich Place                 Diner   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0             Fish Market         Fis

In [13]:
# set number of clusters
kclusters = 10

sfl_grouped_clustering = sfl_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sfl_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])


[5 0 0 0 0 0 0 0 0 0]


In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sfl_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sfl_merged = sfl_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='primary_city')

sfl_merged['Cluster Labels'] = sfl_merged['Cluster Labels'].fillna(0.0).astype(int)

print(sfl_merged.head())

     zip primary_city              acceptable_cities             county  \
0  33004        Dania                    Dania Beach     Broward County   
1  33009   Hallandale  Hallandale Beach, Halndle Bch     Broward County   
2  33010      Hialeah                            NaN  Miami-Dade County   
3  33012      Hialeah                            NaN  Miami-Dade County   
4  33013      Hialeah                            NaN  Miami-Dade County   

   latitude  longitude  population  Cluster Labels   1st Most Common Venue  \
0     26.05     -80.14       12800             0.0      Italian Restaurant   
1     25.98     -80.14       31890             0.0  Furniture / Home Store   
2     25.83     -80.28       37940             0.0        Cuban Restaurant   
3     25.86     -80.29       62200             0.0        Cuban Restaurant   
4     25.86     -80.27       29190             0.0        Cuban Restaurant   

       2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0   

In [25]:
address = 'Fort Lauderdale, FL'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical center coordinate of South Florida is {}, {}.'.format(latitude, longitude))

The geograpical center coordinate of South Florida is 26.1223084, -80.1433786.


In [34]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for lat, lon, poi, cluster in zip(sfl_merged['latitude'], sfl_merged['longitude'],
                                  sfl_merged['primary_city'], sfl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters